# Firm Exit Rates and Regulation

In [1]:
# library
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # plot
import plotly.express as px # geographic plot
from plotly.offline import plot # plot geographic maps
import plotly.graph_objects as go
import statsmodels.formula.api as smf # regression with formula
from stargazer.stargazer import Stargazer # latex table
import itertools # iteration pool
from linearmodels.panel import PanelOLS
import lxml
from tabulate import tabulate


In [2]:
# Options
os.chdir("C:\\Users\\zach_\\Desktop\\Research\\BDS")
pd.set_option("display.max.columns", 20)
%matplotlib inline
%load_ext Cython
pd.options.mode.use_inf_as_na = True
pd.options.display.max_columns = 500

# Parameters
save_fig = False

## 1. Secular Trends of Firm Exit Rates 

### a). Conditional on Age

In [100]:
# read bds data
bds_age = pd.read_csv('C:\\Users\\zach_\\Nobackupfile\\Data\\BDS\\bds2019_firm_age.csv')

In [102]:
# clean and construct variables
bds_age['firms'] = pd.to_numeric(bds_age['firms'], errors='coerce', downcast=None)
bds_age['firmdeath_firms'] = pd.to_numeric(bds_age['firmdeath_firms'], errors='coerce', downcast=None)
bds_age['firm_death_rate'] = bds_age['firmdeath_firms']/ bds_age['firms']
bds_age['year'] = pd.to_numeric(bds_age['year'], errors='coerce', downcast=None)

# age group measure
bds_age['entry'] = (bds_age['fage'] == 'b) 1')
bds_age['new'] = (bds_age['fage'] == 'c) 2')|(bds_age['fage'] == 'd) 3')|(bds_age['fage'] == 'e) 4')|(bds_age['fage'] == 'f) 5') 
bds_age['young'] = (bds_age['fage'] == 'g) 6 to 10')
bds_age['old']  = (bds_age['fage'] == 'h) 11 to 15')|(bds_age['fage'] == 'i) 16 to 20')|(bds_age['fage'] == 'j) 21 to 25')|(bds_age['fage'] == 'k) 26+')|(bds_age['fage'] == 'l) Left Censored') 
bds_age['age_gp'] = bds_age['entry']* 1 + bds_age['new']*2 + bds_age['young'] *3 + bds_age['old'] *4

# sample restriction
bds_age = bds_age[bds_age['age_gp'] != 0]

In [106]:
# construct death rate
death_rate_age = pd.DataFrame()
death_rate_age['1985 - 1990'] = bds_age[(bds_age['year']>1985)&(bds_age['year']<1991) ].groupby('age_gp')['firmdeath_firms'].sum() / \
                            bds_age[(bds_age['year']>1985)&(bds_age['year']<1991) ].groupby('age_gp')['firms'].sum()

death_rate_age['2015 - 2019'] = bds_age[(bds_age['year']>2014)&(bds_age['year']<2020) ].groupby('age_gp')['firmdeath_firms'].sum() / \
                            bds_age[(bds_age['year']>2014)&(bds_age['year']<2020) ].groupby('age_gp')['firms'].sum()
death_rate_age['change_rate'] = (death_rate_age['2015 - 2019'] - death_rate_age['1985 - 1990'])/death_rate_age['1985 - 1990']

fig = death_rate_age.plot.bar(title='Exit Rates by Ages')
fig.set_xlabel("Age Groups")
fig.set_ylabel("Exit Rates")
fig = fig.get_figure()

if save_fig:
    fig.savefig('exit_age.pdf')

In [108]:
# death rate time series
firm_death_rate_entry = bds_age[bds_age['entry']].groupby('year')['firmdeath_firms'].sum() / bds_age[bds_age['entry']].groupby('year')['firms'].sum()
firm_death_rate_new = bds_age[bds_age['new']].groupby('year')['firmdeath_firms'].sum() / bds_age[bds_age['new']].groupby('year')['firms'].sum()
firm_death_rate_young = bds_age[bds_age['young']].groupby('year')['firmdeath_firms'].sum() / bds_age[bds_age['young']].groupby('year')['firms'].sum()
firm_death_rate_old = bds_age[bds_age['old']].groupby('year')['firmdeath_firms'].sum() / bds_age[bds_age['old']].groupby('year')['firms'].sum()

# plot
plt.plot(firm_death_rate_entry)
plt.plot(firm_death_rate_new)
plt.plot(firm_death_rate_young)
plt.plot(firm_death_rate_old)
plt.xlabel('Year')
plt.ylabel('Exit Rates')
plt.legend(['New','Young','Mature','Old'])

if save_fig:
    plt.savefig('exit_age_time.pdf')

### b) conditional on sizes

In [110]:
# load bds data
bds_size = pd.read_csv('C:\\Users\\zach_\\Nobackupfile\\Data\\BDS\\bds2019_firm_size.csv')

In [112]:
# clean and construct variables
bds_size['firms'] = pd.to_numeric(bds_size['firms'], errors='coerce', downcast=None)
bds_size['firmdeath_firms'] = pd.to_numeric(bds_size['firmdeath_firms'], errors='coerce', downcast=None)
bds_size['firm_death_rate'] = bds_size['firmdeath_firms']/ bds_size['firms']
bds_size['year'] = pd.to_numeric(bds_size['year'], errors='coerce', downcast=None)

In [113]:
# construct death rate
death_rate_size = pd.DataFrame()
death_rate_size['1985 - 1990'] = bds_size[(bds_size['year']>1985)&(bds_size['year']<1991) ].groupby('fsize')['firmdeath_firms'].sum() / \
                                bds_size[(bds_size['year']>1985)&(bds_size['year']<1991) ].groupby('fsize')['firms'].sum()

death_rate_size['2015 - 2019'] = bds_size[(bds_size['year']>2014)&(bds_size['year']<2020) ].groupby('fsize')['firmdeath_firms'].sum() / \
                                bds_size[(bds_size['year']>2014)&(bds_size['year']<2020) ].groupby('fsize')['firms'].sum()
death_rate_size['change_rate'] = (death_rate_size['2015 - 2019'] - death_rate_size['1985 - 1990'])/death_rate_size['1985 - 1990']/10

fig = death_rate_size.plot.bar(title='Exit Rates by Size')
fig.set_xlabel("Size Groups")
fig.set_ylabel("Exit Rates")
fig = fig.get_figure()

if save_fig:
    fig.savefig('exit_size.pdf')

### c) Conditional on sector

In [173]:
# load bds data
bds_sector = pd.read_csv('C:\\Users\\zach_\\Nobackupfile\\Data\\BDS\\bds2019_sector.csv')

In [117]:
# clean and construct variables
bds_sector['firms'] = pd.to_numeric(bds_sector['firms'], errors='coerce', downcast=None)
bds_sector['firmdeath_firms'] = pd.to_numeric(bds_sector['firmdeath_firms'], errors='coerce', downcast=None)
bds_sector['firm_death_rate'] = bds_sector['firmdeath_firms']/ bds_sector['firms']
bds_sector['year'] = pd.to_numeric(bds_sector['year'], errors='coerce', downcast=None)

In [118]:
# construct death rate
death_rate_sector = pd.DataFrame()
death_rate_sector['1985 - 1990'] = bds_sector[(bds_sector['year']>1985)&(bds_sector['year']<1991) ].groupby('sector')['firmdeath_firms'].sum() / \
                                bds_sector[(bds_sector['year']>1985)&(bds_sector['year']<1991) ].groupby('sector')['firms'].sum()

death_rate_sector['2015 - 2019'] = bds_sector[(bds_sector['year']>2014)&(bds_sector['year']<2020) ].groupby('sector')['firmdeath_firms'].sum() / \
                                bds_sector[(bds_sector['year']>2014)&(bds_sector['year']<2020) ].groupby('sector')['firms'].sum()
death_rate_sector['change_rate'] = (death_rate_sector['2015 - 2019'] - death_rate_sector['1985 - 1990'])/death_rate_sector['1985 - 1990']/10

fig = death_rate_sector.plot.bar(title='Exit Rates by Sector')
fig.set_xlabel("Sectors")
fig.set_ylabel("Exit Rates")
fig = fig.get_figure()

if save_fig:
    fig.savefig('exit_sector.pdf')

### d) Conditional on initial

In [177]:
# load bds data
bds_init_size = pd.read_csv('C:\\Users\\zach_\\Nobackupfile\\Data\\BDS\\bds2019_initial_size.csv')

In [123]:
# clean and construct variables
bds_init_size['firms'] = pd.to_numeric(bds_init_size['firms'], errors='coerce', downcast=None)
bds_init_size['firmdeath_firms'] = pd.to_numeric(bds_init_size['firmdeath_firms'], errors='coerce', downcast=None)
bds_init_size['firm_death_rate'] = bds_init_size['firmdeath_firms']/ bds_sector['firms']
bds_init_size['year'] = pd.to_numeric(bds_init_size['year'], errors='coerce', downcast=None)

In [124]:
# construct death rate
death_rate_init_size = pd.DataFrame()
death_rate_init_size['1985 - 1990'] = bds_init_size[(bds_init_size['year']>1985)&(bds_init_size['year']<1991) ].groupby('ifsize')['firmdeath_firms'].sum() / \
                                bds_init_size[(bds_init_size['year']>1985)&(bds_init_size['year']<1991) ].groupby('ifsize')['firms'].sum()

death_rate_init_size['2015 - 2019'] = bds_init_size[(bds_init_size['year']>2014)&(bds_init_size['year']<2020) ].groupby('ifsize')['firmdeath_firms'].sum() / \
                                bds_init_size[(bds_init_size['year']>2014)&(bds_init_size['year']<2020) ].groupby('ifsize')['firms'].sum()
death_rate_init_size['change_rate'] = (death_rate_init_size['2015 - 2019'] - death_rate_init_size['1985 - 1990'])/death_rate_init_size['1985 - 1990']/10

fig = death_rate_init_size.plot.bar(title='Exit Rates by Init Size')
fig.set_xlabel("Init Size")
fig.set_ylabel("Exit Rates")
fig = fig.get_figure()

if save_fig:
    fig.savefig('exit_init_size.pdf')

### e) Conditional on state

In [125]:
# load bds data
bds_state = pd.read_csv('C:\\Users\\zach_\\Nobackupfile\\Data\\BDS\\bds2019_state.csv')

In [127]:
# clean and construct variables
bds_state['firms'] = pd.to_numeric(bds_state['firms'], errors='coerce', downcast=None)
bds_state['firmdeath_firms'] = pd.to_numeric(bds_state['firmdeath_firms'], errors='coerce', downcast=None)
bds_state['firm_death_rate'] = bds_state['firmdeath_firms']/ bds_sector['firms']
bds_state['year'] = pd.to_numeric(bds_state['year'], errors='coerce', downcast=None)

In [128]:
# construct death rate
death_rate_state = pd.DataFrame()
death_rate_state['1985 - 1990'] = bds_state[(bds_state['year']>1985)&(bds_state['year']<1991) ].groupby('st')['firmdeath_firms'].sum() / \
                                bds_state[(bds_state['year']>1985)&(bds_state['year']<1991) ].groupby('st')['firms'].sum()

death_rate_state['2015 - 2019'] = bds_state[(bds_state['year']>2014)&(bds_state['year']<2020) ].groupby('st')['firmdeath_firms'].sum() / \
                                bds_state[(bds_state['year']>2014)&(bds_state['year']<2020) ].groupby('st')['firms'].sum()
death_rate_state['change_rate'] = (death_rate_state['2015 - 2019'] - death_rate_state['1985 - 1990'])/death_rate_state['1985 - 1990']*100

# merge state code
state_code = pd.read_csv('C:\\Users\\zach_\\Nobackupfile\\Data\\UI\\STATE_CODE.csv')
death_rate_state = death_rate_state.merge(state_code, on = "st")

In [4]:
# plot geographic distribution - positive federal loans
fig1 = px.choropleth(death_rate_state,
                    locations='ST', 
                    locationmode="USA-states", 
                    color='change_rate',
                    color_continuous_scale="Viridis_r", 
                    scope="usa",
                    labels={"with_fed_loans": "# of Fed Loans"})
fig1.show()

if save_fig:
    fig1.write_image("exit_state.png")

NameError: name 'death_rate_state' is not defined

## 2. Replicate Results

### a) Aggregation change on both measures
This section intends to replicate the results from the previous literature regarding the impacts of Regdata on Firm birth
 - Effects from the levels of aggregation
 - Flows (startup rates) vs magnitude (number of the startups)
 - Effects on exit rates after controls for startup
 - Compared with results based on ages

In [50]:
# create data at different aggregation
data_dic = dict()

# reg data
regdata = pd.read_csv('C:\\Users\\zach_\\Nobackupfile\\Data\\BDS\\Regdata\\regdata_4_0_industries.csv')
regdata['sector'] = regdata['NAICS']

for naics in range(2, 5):
    # load dataset
    file_name = f"bds2019_naics_{naics}_age"
    file_address = f"C:\\Users\\zach_\\Nobackupfile\\Data\\BDS\\{file_name}.csv"
    df = pd.read_csv(file_address)
    if naics == 2: # convert state string to numeric
        df['sector'] = pd.to_numeric(df['sector'].str.slice(0,2), errors='coerce', downcast=None)
    
    # clean and create variables
    df['firms'] = pd.to_numeric(df['firms'], errors='coerce', downcast=None)
    df['firmdeath_firms'] = pd.to_numeric(df['firmdeath_firms'], errors='coerce', downcast=None)
    
    # aggregation for entry and death measure
    df_age = pd.DataFrame()
    df_age['entry'] = df.loc[df['fage'] == 'a) 0', :].groupby(['year', 'sector'])['firms'].sum()
    df_age['incumbents'] = df.loc[df['fage'] != 'a) 0', :].groupby(['year', 'sector'])['firms'].sum()
    df_age['death'] = df.loc[df['fage'] != 'a) 0', :].groupby(['year', 'sector'])['firmdeath_firms'].sum()

    # merge 
    df_age_merge= df_age.merge(regdata, how = "left", left_on=["year", "sector"], right_on=["year", "sector"])

    # create some variables
    df_age_merge['log_restriction_1'] = np.log(df_age_merge['industry_restrictions_1_0'])
    df_age_merge['log_restriction_2'] = np.log(df_age_merge['industry_restrictions_2_0'])
    df_age_merge['log_entry'] = np.log(df_age_merge['entry'])
    df_age_merge['log_death'] = np.log(df_age_merge['death'])
    df_age_merge['log_incumbents'] = np.log(df_age_merge['incumbents'])
    df_age_merge['entry_rate'] = df_age_merge['entry']/df_age_merge['incumbents'] 
    df_age_merge['death_rate'] = df_age_merge['death']/df_age_merge['incumbents'] 
    
    data_dic[file_name] = df_age_merge

c:\Users\zach_\anaconda3\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning:

divide by zero encountered in log



#### The following regressions are examined:

1. Check the impacts of the level of aggregation
   $${\sf entry\_rate} _{s,t} = \alpha_s + \gamma_t + \beta_1 {\sf Reg}_{s,t} + \epsilon_{s,t}$$

In [51]:
results_dict_1 = dict()
for naics_curr in range(2, 5):
    # load data
    file_name_curr = f"bds2019_naics_{naics_curr}_age"
    data = data_dic[file_name_curr]
    # sample restriction
    data = data[data.year > 1985]

    # regression
    data = data.set_index(['sector', 'year'])
    data = data.loc[:, ['entry_rate', 'death_rate', 'log_restriction_1']].dropna()
    mod = PanelOLS.from_formula(formula = 'entry_rate ~ log_restriction_1  + EntityEffects + TimeEffects', data = data)
    res = mod.fit(cov_type = 'heteroskedastic')

    # results
    results_as_html = res.summary.tables[1].as_html()
    results_table = pd.read_html(results_as_html, header=0, index_col=0)[0]
    results_dict_1[file_name_curr] = results_table

print(results_dict_1['bds2019_naics_2_age'])
print(results_dict_1['bds2019_naics_3_age'])
print(results_dict_1['bds2019_naics_4_age'])

                   Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
log_restriction_1     0.0051      0.004  1.2793   0.2015   -0.0027    0.0128
                   Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
log_restriction_1      0.002     0.0026  0.7752   0.4383   -0.0031    0.0071
                   Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
log_restriction_1     0.0049     0.0013  3.7107   0.0002    0.0023    0.0075


2. Impacts of Death Rates
   $${\sf entry\_rate} _{s,t} = \alpha_s + \gamma_t + \beta_1 {\sf Reg}_{s,t} + \beta_2 {\sf exit\_rates} + \epsilon_{s,t}$$

In [166]:
results_dict_1 = dict()
for naics_curr in range(2, 5):
    # load data
    file_name_curr = f"bds2019_naics_{naics_curr}_age"
    data = data_dic[file_name_curr]
    # sample restriction
    data = data[data.year > 1985]

    # regression
    data = data.set_index(['sector', 'year'])
    data = data.loc[:, ['entry_rate', 'death_rate', 'log_restriction_1']].dropna()
    mod = PanelOLS.from_formula(formula = 'entry_rate ~ log_restriction_1  + death_rate + EntityEffects + TimeEffects', data = data)
    res = mod.fit(cov_type = 'heteroskedastic')

    # results
    results_as_html = res.summary.tables[1].as_html()
    results_table = pd.read_html(results_as_html, header=0, index_col=0)[0]
    results_dict_1[file_name_curr] = results_table

print(results_dict_1['bds2019_naics_2_age'])
print(results_dict_1['bds2019_naics_3_age'])
print(results_dict_1['bds2019_naics_4_age'])

                   Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
death_rate            0.1498     0.0954  1.5702   0.1171   -0.0377    0.3373
log_restriction_1     0.0029     0.0038  0.7739   0.4394   -0.0045    0.0103
                   Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
death_rate            0.7946     0.1394  5.7003   0.0000    0.5212    1.0681
log_restriction_1    -0.0019     0.0027 -0.6960   0.4866   -0.0073    0.0035
                   Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
death_rate            0.0854     0.0449  1.9008   0.0574   -0.0027    0.1734
log_restriction_1     0.0044     0.0013  3.2895   0.0010    0.0018    0.0071


3. using log level measure
   $$\log({\sf entry}_{s,t}) = \alpha_s + \gamma_t + \beta_1 {\sf Reg}_{s,t} + \beta_2 \log({\sf incumbents})  + \epsilon_{s,t}$$

In [167]:
results_dict_3 = dict()
for naics_curr in range(2, 5):
    # load data
    file_name_curr = f"bds2019_naics_{naics_curr}_age"
    data = data_dic[file_name_curr]
    # sample restriction
    data = data[data.year > 1985]

    # regression
    data = data.set_index(['sector', 'year'])
    data = data.loc[:, ['log_entry', 'log_death', 'log_incumbents', 'log_restriction_1']].dropna()
    mod = PanelOLS.from_formula(formula = 'log_entry ~ log_restriction_1 + log_incumbents + EntityEffects + TimeEffects', data = data)
    res = mod.fit(cov_type = 'heteroskedastic')

    # results
    results_as_html = res.summary.tables[1].as_html()
    results_table = pd.read_html(results_as_html, header=0, index_col=0)[0]
    results_dict_3[file_name_curr] = results_table

print(results_dict_3['bds2019_naics_2_age'])
print(results_dict_3['bds2019_naics_3_age'])
print(results_dict_3['bds2019_naics_4_age'])

4. Adding measure of death
   $$\log({\sf entry}_{s,t}) = \alpha_s + \gamma_t + \beta_1 {\sf Reg}_{s,t} + \beta_2 \log({\sf exit})  + \beta_3 \log({\sf incumbents}) + \epsilon_{s,t}$$

In [169]:
results_dict_4 = dict()
for naics_curr in range(2, 5):
    # load data
    file_name_curr = f"bds2019_naics_{naics_curr}_age"
    data = data_dic[file_name_curr]
    # sample restriction
    data = data[data.year > 1985]

    # regression
    data = data.set_index(['sector', 'year'])
    data = data.loc[:, ['log_entry', 'log_death', 'log_incumbents', 'log_restriction_1']].dropna()
    mod = PanelOLS.from_formula(formula = 'log_entry ~ log_restriction_1 + log_death + log_incumbents + EntityEffects + TimeEffects', data = data)
    res = mod.fit(cov_type = 'heteroskedastic')

    # results
    results_as_html = res.summary.tables[1].as_html()
    results_table = pd.read_html(results_as_html, header=0, index_col=0)[0]
    results_dict_4[file_name_curr] = results_table

print(results_dict_4['bds2019_naics_2_age'])
print(results_dict_4['bds2019_naics_3_age'])
print(results_dict_4['bds2019_naics_4_age'])

                   Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
log_death            -0.0035     0.0902 -0.0384   0.9694   -0.1807    0.1738
log_incumbents        1.1326     0.1168  9.6999   0.0000    0.9031    1.3621
log_restriction_1    -0.1198     0.0409 -2.9309   0.0036   -0.2001   -0.0395
                   Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
log_death             0.4836     0.0575  8.4158   0.0000    0.3709    0.5963
log_incumbents        0.6200     0.0870  7.1301   0.0000    0.4494    0.7906
log_restriction_1    -0.0471     0.0200 -2.3530   0.0188   -0.0864   -0.0078
                   Parameter  Std. Err.   T-stat  P-value  Lower CI  Upper CI
log_death             0.0601     0.0234   2.5661   0.0103    0.0142    0.1060
log_incumbents        0.9997     0.0366  27.3030   0.0000    0.9279    1.0715
log_restriction_1     0.0235     0.0137   1.7168   0.0861   -0.0033    0.0503


### a) Aggregation change on Reg data only

In [52]:
# create data at different aggregation
data_dic = dict()

# reg data
regdata = pd.read_csv('C:\\Users\\zach_\\Nobackupfile\\Data\\BDS\\Regdata\\regdata_4_0_industries.csv')
regdata['sector_reg'] = regdata['NAICS']
regdata = regdata.loc[:,["year", "sector_reg", "industry_restrictions_1_0", "industry_restrictions_2_0"]]

# load dataset
file_name = "bds2019_naics_4_age"
file_address = f"C:\\Users\\zach_\\Nobackupfile\\Data\\BDS\\{file_name}.csv"
df = pd.read_csv(file_address)

# clear data
df['firms'] = pd.to_numeric(df['firms'], errors='coerce', downcast=None)
df['firmdeath_firms'] = pd.to_numeric(df['firmdeath_firms'], errors='coerce', downcast=None)

# aggregation for entry and death measure
df_age = pd.DataFrame()
df_age['entry'] = df.loc[df['fage'] == 'a) 0', :].groupby(['year', 'sector'])['firms'].sum()
df_age['incumbents'] = df.loc[df['fage'] != 'a) 0', :].groupby(['year', 'sector'])['firms'].sum()
df_age['death'] = df.loc[df['fage'] != 'a) 0', :].groupby(['year', 'sector'])['firmdeath_firms'].sum()
df_age = df_age.reset_index()

for naics in range(2, 5):
    # clean and create variables
    sector_name = f"sector_{naics}"
    df_age[sector_name] = df_age['sector'].astype(str).str.slice(0,naics)
    df_age[sector_name] = pd.to_numeric(df_age[sector_name])

    df_age = df_age.merge(regdata, how = "left", left_on=["year", sector_name], right_on=["year", "sector_reg"])
    df_age = df_age.rename(columns={"industry_restrictions_1_0": f"industry_restrictions_1_0_{naics}", "industry_restrictions_2_0": f"industry_restrictions_2_0_{naics}"}, errors="raise")
    df_age = df_age.drop(columns='sector_reg')
    df_age[f'log_restriction_1_{naics}'] = np.log(df_age[f'industry_restrictions_1_0_{naics}'])
    df_age[f'log_restriction_2_{naics}'] = np.log(df_age[f'industry_restrictions_2_0_{naics}'])

# create some variables

df_age['log_entry'] = np.log(df_age['entry'])
df_age['log_death'] = np.log(df_age['death'])
df_age['log_incumbents'] = np.log(df_age['incumbents'])
df_age['entry_rate'] = df_age['entry']/df_age['incumbents'] 
df_age['death_rate'] = df_age['death']/df_age['incumbents'] 


c:\Users\zach_\anaconda3\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning:

divide by zero encountered in log



1. Check the impacts of the level of aggregation
   $${\sf entry\_rate}_{naics_4, t} = \alpha_{naics_4} + \gamma_t + \beta_1 {\sf Reg}_{s,t} + \epsilon_{naics_4, t}$$

In [53]:
results_dict_1 = dict()
for naics_curr in range(2, 5):
    # load data
    data = df_age
    # sample restriction
    data = data[data.year > 1985]

    # regression
    data = data.set_index(['sector', 'year'])
    data = data.loc[:, ['entry_rate', 'death_rate', 'log_incumbents', f'log_restriction_1_{naics_curr}']].dropna()
    mod = PanelOLS.from_formula(formula = f'entry_rate ~ log_restriction_1_{naics_curr}  + EntityEffects + TimeEffects', data = data)
    res = mod.fit(cov_type = 'heteroskedastic')

    # results
    results_as_html = res.summary.tables[1].as_html()
    results_table = pd.read_html(results_as_html, header=0, index_col=0)[0]
    file_name_curr = f"reg_{naics_curr}"
    results_dict_1[file_name_curr] = results_table

print(results_dict_1['reg_2'])
print(results_dict_1['reg_3'])
print(results_dict_1['reg_4'])


                     Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
log_restriction_1_2     0.0157     0.0045  3.4732   0.0005    0.0068    0.0246
                     Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
log_restriction_1_3    -0.0024     0.0014 -1.7016   0.0889   -0.0051    0.0004
                     Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
log_restriction_1_4     0.0049     0.0013  3.7107   0.0002    0.0023    0.0075


2. Impacts of Death Rates
   $${\sf entry\_rate} _{naics_4,t} = \alpha_s + \gamma_t + \beta_1 {\sf Reg}_{s,t} + \beta_2 {\sf exit\_rates} + \epsilon_{naics_4,t}$$

In [54]:
results_dict_2 = dict()
for naics_curr in range(2, 5):
    # load data
    data = df_age
    # sample restriction
    data = data[data.year > 1985]

    # regression
    data = data.set_index(['sector', 'year'])
    data = data.loc[:, ['entry_rate', 'death_rate', 'log_incumbents', f'log_restriction_1_{naics_curr}']].dropna()
    mod = PanelOLS.from_formula(formula = f'entry_rate ~ log_restriction_1_{naics_curr}  + death_rate + EntityEffects + TimeEffects',
                                data = data)
    res = mod.fit(cov_type = 'heteroskedastic')

    # results
    results_as_html = res.summary.tables[1].as_html()
    results_table = pd.read_html(results_as_html, header=0, index_col=0)[0]
    file_name_curr = f"reg_{naics_curr}"
    results_dict_2[file_name_curr] = results_table

print(results_dict_2['reg_2'])
print(results_dict_2['reg_3'])
print(results_dict_2['reg_4'])

                     Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
death_rate              0.3942     0.0898  4.3900   0.0000    0.2182    0.5702
log_restriction_1_2     0.0128     0.0044  2.9323   0.0034    0.0043    0.0214
                     Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
death_rate              0.3661     0.0760  4.8183   0.0000    0.2171    0.5151
log_restriction_1_3    -0.0035     0.0014 -2.4994   0.0125   -0.0063   -0.0008
                     Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
death_rate              0.0854     0.0449  1.9008   0.0574   -0.0027    0.1734
log_restriction_1_4     0.0044     0.0013  3.2895   0.0010    0.0018    0.0071


3. using log level measure
   $$\log({\sf entry}_{naics_4,t}) = \alpha_s + \gamma_t + \beta_1 {\sf Reg}_{s,t} + \beta_2 \log({\sf incumbents})  + \epsilon_{naics_4,t}$$

In [253]:
results_dict_3 = dict()
for naics_curr in range(2, 5):
    # load data
    data = df_age
    # sample restriction
    data = data[data.year > 1985]

    # regression
    data = data.set_index(['sector', 'year'])
    data = data.loc[:, ['log_entry', 'log_death', 'log_incumbents', f'log_restriction_1_{naics_curr}']].dropna()
    mod = PanelOLS.from_formula(formula = f'log_entry ~ log_restriction_1_{naics_curr}  + log_incumbents + EntityEffects + TimeEffects',
                                data = data)
    res = mod.fit(cov_type = 'heteroskedastic')

    # results
    results_as_html = res.summary.tables[1].as_html()
    results_table = pd.read_html(results_as_html, header=0, index_col=0)[0]
    file_name_curr = f"reg_{naics_curr}"
    results_dict_3[file_name_curr] = results_table

print(results_dict_3['reg_2'])
print(results_dict_3['reg_3'])
print(results_dict_3['reg_4'])

                     Parameter  Std. Err.   T-stat  P-value  Lower CI  \
log_incumbents          0.8967     0.0262  34.2530    0.000    0.8454   
log_restriction_1_2    -0.0342     0.0309  -1.1077    0.268   -0.0948   

                     Upper CI  
log_incumbents         0.9480  
log_restriction_1_2    0.0264  
                     Parameter  Std. Err.   T-stat  P-value  Lower CI  \
log_incumbents          0.9663     0.0272  35.5790      0.0    0.9130   
log_restriction_1_3    -0.0626     0.0123  -5.1111      0.0   -0.0867   

                     Upper CI  
log_incumbents         1.0195  
log_restriction_1_3   -0.0386  
                     Parameter  Std. Err.   T-stat  P-value  Lower CI  \
log_incumbents          1.0677     0.0258  41.3390   0.0000    1.0171   
log_restriction_1_4     0.0267     0.0137   1.9478   0.0515   -0.0002   

                     Upper CI  
log_incumbents         1.1183  
log_restriction_1_4    0.0536  


4. Adding measure of death
   $$\log({\sf entry}_{naics_4,t}) = \alpha_s + \gamma_t + \beta_1 {\sf Reg}_{s,t} + \beta_2 \log({\sf exit})  + \beta_3 \log({\sf incumbents}) + \epsilon_{naics_4,t}$$

In [7]:
results_dict_4 = dict()
for naics_curr in range(2, 5):
    # load data
    data = df_age
    # sample restriction
    data = data[data.year > 1985]

    # regression
    data = data.set_index(['sector', 'year'])
    data = data.loc[:, ['log_entry', 'log_death', 'log_incumbents', f'log_restriction_1_{naics_curr}']].dropna()
    mod = PanelOLS.from_formula(formula = f'log_entry ~ log_restriction_1_{naics_curr}  + log_incumbents + log_death + EntityEffects + TimeEffects',
                                data = data)
    res = mod.fit(cov_type = 'heteroskedastic')

    # results
    results_as_html = res.summary.tables[1].as_html()
    results_table = pd.read_html(results_as_html, header=0, index_col=0)[0]
    file_name_curr = f"reg_{naics_curr}"
    results_dict_4[file_name_curr] = results_table

print(results_dict_4['reg_2'])
print(results_dict_4['reg_3'])
print(results_dict_4['reg_4'])

                     Parameter  Std. Err.   T-stat  P-value  Lower CI  \
log_death               0.1972     0.0268   7.3552   0.0000    0.1446   
log_incumbents          0.7018     0.0398  17.6560   0.0000    0.6239   
log_restriction_1_2    -0.0344     0.0291  -1.1809   0.2377   -0.0915   

                     Upper CI  
log_death              0.2497  
log_incumbents         0.7798  
log_restriction_1_2    0.0227  
                     Parameter  Std. Err.   T-stat  P-value  Lower CI  \
log_death               0.1683     0.0259   6.4963      0.0    0.1175   
log_incumbents          0.8016     0.0412  19.4790      0.0    0.7209   
log_restriction_1_3    -0.0678     0.0123  -5.5352      0.0   -0.0919   

                     Upper CI  
log_death              0.2191  
log_incumbents         0.8823  
log_restriction_1_3   -0.0438  
                     Parameter  Std. Err.   T-stat  P-value  Lower CI  \
log_death               0.0601     0.0234   2.5661   0.0103    0.0142   
log_incumben

2. All levels
   $${\sf entry\_rate} _{naics_4,t} = \alpha_s + \gamma_t + \beta_1 {\sf Reg}_{naics_2,t} + \beta_2 {\sf Reg}_{naics_3,t} + \beta_3 {\sf Reg}_{naics_4,t} + \beta_4 {\sf exit\_rates} + \epsilon_{naics_4,t}$$

In [61]:
data = df_age
# sample restriction
data = data[data.year > 1985]

# regression
data = data.set_index(['sector', 'year'])
data = data.loc[:, ['entry_rate', 'death_rate', 'log_incumbents',
                    'log_restriction_1_2', 'log_restriction_1_3', 
                    'log_restriction_1_4']].dropna()
mod = PanelOLS.from_formula(formula = 
                            'entry_rate ~ log_restriction_1_2 + log_restriction_1_3 + log_restriction_1_4  \
                            + death_rate  + EntityEffects + TimeEffects',
                            data = data)
res = mod.fit(cov_type = 'heteroskedastic')

# results
results_as_html = res.summary.tables[1].as_html()
results_table = pd.read_html(results_as_html, header=0, index_col=0)[0]

print(results_table)

                     Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
death_rate              0.0822     0.0552  1.4895   0.1365   -0.0260    0.1905
log_restriction_1_2     0.0043     0.0049  0.8673   0.3859   -0.0054    0.0140
log_restriction_1_3    -0.0206     0.0044 -4.6937   0.0000   -0.0292   -0.0120
log_restriction_1_4     0.0076     0.0043  1.7850   0.0744   -0.0008    0.0160


## 3. Exit Rates and Reg Data

This section uses previous specification to study the effects of regulation on firm exit rates.
Several key issues are examinated here:
1. What is the effects of the regulation on firm exit rates?
2. Are there some issues regarding aggregation levels?
3. Are results robust using different classifications of the BDS data?

In [8]:
# Using the precedure from the previous section to construct data
# create data at different aggregation
data_dic = dict()

# reg data
regdata = pd.read_csv('C:\\Users\\zach_\\Nobackupfile\\Data\\BDS\\Regdata\\regdata_4_0_industries.csv')
regdata['sector_reg'] = regdata['NAICS']
regdata = regdata.loc[:,["year", "sector_reg", "industry_restrictions_1_0", "industry_restrictions_2_0"]]

# load dataset
file_name = "bds2019_naics_4_age"
file_address = f"C:\\Users\\zach_\\Nobackupfile\\Data\\BDS\\{file_name}.csv"
df = pd.read_csv(file_address)

# clear data
df['firms'] = pd.to_numeric(df['firms'], errors='coerce', downcast=None)
df['firmdeath_firms'] = pd.to_numeric(df['firmdeath_firms'], errors='coerce', downcast=None)

# aggregation for entry and death measure
df_age = pd.DataFrame()
df_age['entry'] = df.loc[df['fage'] == 'a) 0', :].groupby(['year', 'sector'])['firms'].sum()
df_age['incumbents'] = df.loc[df['fage'] != 'a) 0', :].groupby(['year', 'sector'])['firms'].sum()
df_age['death'] = df.loc[df['fage'] != 'a) 0', :].groupby(['year', 'sector'])['firmdeath_firms'].sum()
df_age = df_age.reset_index()

for naics in range(2, 5):
    # clean and create variables
    sector_name = f"sector_{naics}"
    df_age[sector_name] = df_age['sector'].astype(str).str.slice(0,naics)
    df_age[sector_name] = pd.to_numeric(df_age[sector_name])

    df_age = df_age.merge(regdata, how = "left", left_on=["year", sector_name], right_on=["year", "sector_reg"])
    df_age = df_age.rename(columns={"industry_restrictions_1_0": f"industry_restrictions_1_0_{naics}", "industry_restrictions_2_0": f"industry_restrictions_2_0_{naics}"}, errors="raise")
    df_age = df_age.drop(columns='sector_reg')
    df_age[f'log_restriction_1_{naics}'] = np.log(df_age[f'industry_restrictions_1_0_{naics}'])
    df_age[f'log_restriction_2_{naics}'] = np.log(df_age[f'industry_restrictions_2_0_{naics}'])

# create some variables

df_age['log_entry'] = np.log(df_age['entry'])
df_age['log_death'] = np.log(df_age['death'])
df_age['log_incumbents'] = np.log(df_age['incumbents'])
df_age['entry_rate'] = df_age['entry']/df_age['incumbents'] 
df_age['death_rate'] = df_age['death']/df_age['incumbents'] 

C:\Users\zach_\anaconda3\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning:

divide by zero encountered in log



1. Check the impacts of the level of aggregation
   1. one level reg only
   $${\sf Exit\_rate}_{naics_4, t} = \alpha_{naics_4} + \gamma_t + \beta_1 {\sf Reg}_{s,t} + \epsilon_{naics_4, t}$$

In [10]:
results_dict_1 = dict()
for naics_curr in range(2, 5):
    # load data
    data = df_age
    # sample restriction
    data = data[data.year > 1985]

    # regression
    data = data.set_index(['sector', 'year'])
    data = data.loc[:, ['entry_rate', 'death_rate', 'log_incumbents', f'log_restriction_1_{naics_curr}']].dropna()
    mod = PanelOLS.from_formula(formula = f'death_rate ~ log_restriction_1_{naics_curr}  + EntityEffects + TimeEffects', data = data)
    res = mod.fit(cov_type = 'heteroskedastic')

    # results
    results_as_html = res.summary.tables[1].as_html()
    results_table = pd.read_html(results_as_html, header=0, index_col=0)[0]
    file_name_curr = f"reg_{naics_curr}"
    results_dict_1[file_name_curr] = results_table

print(results_dict_1['reg_2'])
print(results_dict_1['reg_3'])
print(results_dict_1['reg_4'])


                     Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
log_restriction_2_2     0.0076     0.0035  2.1743   0.0297    0.0008    0.0145
                     Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
log_restriction_2_3     0.0033     0.0012  2.7036   0.0069    0.0009    0.0056
                     Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
log_restriction_2_4     0.0054     0.0012  4.3824      0.0     0.003    0.0078


   2. all levels
   $${\sf Exit\_rate}_{naics_4, t} = \alpha_{naics_4} + \gamma_t + \beta_1 {\sf Reg}_{naics_2,t} + \beta_2 {\sf Reg}_{naics_3,t} + \beta_3 {\sf Reg}_{naics_4,t} + \epsilon_{naics_4, t}$$

In [14]:
data = df_age
# sample restriction
data = data[data.year > 1985]

# regression
data = data.set_index(['sector', 'year'])
data = data.loc[:, ['entry_rate', 'death_rate', 'log_incumbents', 
                    'log_restriction_1_2', 'log_restriction_1_3', 
                    'log_restriction_1_4']].dropna()
mod = PanelOLS.from_formula(formula = 
                            'death_rate ~ log_restriction_1_2 + log_restriction_1_3 + log_restriction_1_4  \
                            + EntityEffects + TimeEffects',
                            data = data)
res = mod.fit(cov_type = 'heteroskedastic')

# results
results_as_html = res.summary.tables[1].as_html()
results_table = pd.read_html(results_as_html, header=0, index_col=0)[0]

print(results_table)


                     Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
log_restriction_1_2    -0.0017     0.0038 -0.4419   0.6586   -0.0090    0.0057
log_restriction_1_3    -0.0016     0.0047 -0.3408   0.7333   -0.0107    0.0076
log_restriction_1_4     0.0073     0.0036  2.0306   0.0424    0.0002    0.0143


2. Impacts of Entry Rates
   1. one level of regulation
   $${\sf Exit\_rate} _{naics_4,t} = \alpha_s + \gamma_t + \beta_1 {\sf Reg}_{s,t} + \beta_2 {\sf entry\_rates} + \epsilon_{naics_4,t}$$

In [11]:
results_dict_2 = dict()
for naics_curr in range(2, 5):
    # load data
    data = df_age
    # sample restriction
    data = data[data.year > 1985]

    # regression
    data = data.set_index(['sector', 'year'])
    data = data.loc[:, ['entry_rate', 'death_rate', 'log_incumbents', f'log_restriction_1_{naics_curr}']].dropna()
    mod = PanelOLS.from_formula(formula = f'death_rate ~ log_restriction_1_{naics_curr}  + entry_rate + EntityEffects + TimeEffects',
                                data = data)
    res = mod.fit(cov_type = 'heteroskedastic')

    # results
    results_as_html = res.summary.tables[1].as_html()
    results_table = pd.read_html(results_as_html, header=0, index_col=0)[0]
    file_name_curr = f"reg_{naics_curr}"
    results_dict_2[file_name_curr] = results_table

print(results_dict_2['reg_2'])
print(results_dict_2['reg_3'])
print(results_dict_2['reg_4'])

                     Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
entry_rate              0.2053     0.0700  2.9348   0.0034    0.0682    0.3424
log_restriction_1_2     0.0041     0.0038  1.0816   0.2795   -0.0033    0.0115
                     Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
entry_rate              0.2154     0.0539  3.9972   0.0001    0.1098    0.3211
log_restriction_1_3     0.0036     0.0012  3.0014   0.0027    0.0013    0.0060
                     Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
entry_rate              0.0572     0.0294  1.9477   0.0515   -0.0004    0.1148
log_restriction_1_4     0.0053     0.0012  4.3834   0.0000    0.0029    0.0076


   2. all levels
   $${\sf exit\_rate} _{naics_4,t} = \alpha_s + \gamma_t + \beta_1 {\sf Reg}_{naics_2,t} + \beta_2 {\sf Reg}_{naics_3,t} + \beta_3 {\sf Reg}_{naics_4,t} + \beta_4 {\sf entry\_rates} + \epsilon_{naics_4,t}$$

In [24]:
data = df_age
# sample restriction
data = data[data.year > 1985]

# regression
data = data.set_index(['sector', 'year'])
data = data.loc[:, ['entry_rate', 'death_rate', 'log_incumbents',
                    'log_restriction_1_2', 'log_restriction_1_3', 
                    'log_restriction_1_4']].dropna()
mod = PanelOLS.from_formula(formula = 
                            'death_rate ~ log_restriction_1_2 + log_restriction_1_3 + log_restriction_1_4  \
                            + entry_rate  + EntityEffects + TimeEffects',
                            data = data)
res = mod.fit(cov_type = 'heteroskedastic')

# results
results_as_html = res.summary.tables[1].as_html()
results_table = pd.read_html(results_as_html, header=0, index_col=0)[0]

print(results_table)

                     Parameter  Std. Err.  T-stat  P-value  Lower CI  Upper CI
entry_rate              0.0529     0.0338  1.5646   0.1178 -0.013400    0.1193
log_restriction_1_2    -0.0019     0.0037 -0.5083   0.6113 -0.009100    0.0054
log_restriction_1_3    -0.0005     0.0047 -0.1054   0.9161 -0.009600    0.0087
log_restriction_1_4     0.0068     0.0035  1.9674   0.0493  0.000022    0.0137


3. using log level measure
   1. one level of regulation
   $$\log({\sf exit}_{naics_4,t}) = \alpha_s + \gamma_t + \beta_1 {\sf Reg}_{s,t} + \beta_2 \log({\sf incumbents})  + \epsilon_{naics_4,t}$$

In [15]:
results_dict_3 = dict()
for naics_curr in range(2, 5):
    # load data
    data = df_age
    # sample restriction
    data = data[data.year > 1985]

    # regression
    data = data.set_index(['sector', 'year'])
    data = data.loc[:, ['log_entry', 'log_death', 'log_incumbents', f'log_restriction_1_{naics_curr}']].dropna()
    mod = PanelOLS.from_formula(formula = f'log_death ~ log_restriction_1_{naics_curr}  + log_incumbents + EntityEffects + TimeEffects',
                                data = data)
    res = mod.fit(cov_type = 'heteroskedastic')

    # results
    results_as_html = res.summary.tables[1].as_html()
    results_table = pd.read_html(results_as_html, header=0, index_col=0)[0]
    file_name_curr = f"reg_{naics_curr}"
    results_dict_3[file_name_curr] = results_table

print(results_dict_3['reg_2'])
print(results_dict_3['reg_3'])
print(results_dict_3['reg_4'])

                     Parameter  Std. Err.   T-stat  P-value  Lower CI  \
log_incumbents          0.9882     0.0333  29.6840   0.0000    0.9229   
log_restriction_1_2     0.0009     0.0296   0.0289   0.9769   -0.0572   

                     Upper CI  
log_incumbents         1.0534  
log_restriction_1_2    0.0590  
                     Parameter  Std. Err.   T-stat  P-value  Lower CI  \
log_incumbents          0.9783     0.0278  35.2150   0.0000    0.9238   
log_restriction_1_3     0.0309     0.0118   2.6251   0.0087    0.0078   

                     Upper CI  
log_incumbents         1.0327  
log_restriction_1_3    0.0540  
                     Parameter  Std. Err.   T-stat  P-value  Lower CI  \
log_incumbents          1.1318     0.0249  45.3910      0.0    1.0829   
log_restriction_1_4     0.0533     0.0130   4.1003      0.0    0.0278   

                     Upper CI  
log_incumbents         1.1807  
log_restriction_1_4    0.0788  


   2. all levels of regulation
   $$\log({\sf exit}_{naics_4,t}) = \alpha_s + \gamma_t+ \beta_1 {\sf Reg}_{naics_2,t} + \beta_2 {\sf Reg}_{naics_3,t} + \beta_3 {\sf Reg}_{naics_4,t} + \beta_4 \log({\sf incumbents})  + \epsilon_{naics_4,t}$$

In [49]:
data = df_age
# sample restriction
data = data[data.year > 1985]

# regression
data = data.set_index(['sector', 'year'])
data = data.loc[:, ['log_entry', 'log_death', 'log_incumbents', 
                    'log_restriction_1_2', 'log_restriction_1_3', 
                    'log_restriction_1_4']].dropna()
mod = PanelOLS.from_formula(formula = 
                            'log_death ~ log_restriction_1_2 + log_restriction_1_3 + log_restriction_1_4  \
                            + log_incumbents + EntityEffects + TimeEffects',
                            data = data)
res = mod.fit(cov_type = 'heteroskedastic')

# results
results_as_html = res.summary.tables[1].as_html()
results_table = pd.read_html(results_as_html, header=0, index_col=0)[0]

print(results_table)

KeyError: "None of [Index(['log_entry', 'log_death', 'log_incumbents', 'log_restriction_1_2',\n       'log_restriction_1_3', 'log_restriction_1_4'],\n      dtype='object')] are in the [columns]"

4. Adding measure of death
   1. one level only
   $$\log({\sf exit}_{naics_4,t}) = \alpha_s + \gamma_t + \beta_1 {\sf Reg}_{s,t} + \beta_2 \log({\sf entry})  + \beta_3 \log({\sf incumbents}) + \epsilon_{naics_4,t}$$

In [18]:
results_dict_4 = dict()
for naics_curr in range(2, 5):
    # load data
    data = df_age
    # sample restriction
    data = data[data.year > 1985]

    # regression
    data = data.set_index(['sector', 'year'])
    data = data.loc[:, ['log_entry', 'log_death', 'log_incumbents', f'log_restriction_1_{naics_curr}']].dropna()
    mod = PanelOLS.from_formula(formula = f'log_death ~ log_restriction_1_{naics_curr}  + log_incumbents + log_entry + EntityEffects + TimeEffects',
                                data = data)
    res = mod.fit(cov_type = 'heteroskedastic')

    # results
    results_as_html = res.summary.tables[1].as_html()
    results_table = pd.read_html(results_as_html, header=0, index_col=0)[0]
    file_name_curr = f"reg_{naics_curr}"
    results_dict_4[file_name_curr] = results_table

print(results_dict_4['reg_2'])
print(results_dict_4['reg_3'])
print(results_dict_4['reg_4'])

                     Parameter  Std. Err.   T-stat  P-value  Lower CI  \
log_entry               0.1830     0.0237   7.7205    0.000    0.1366   
log_incumbents          0.8240     0.0440  18.7470    0.000    0.7379   
log_restriction_1_2     0.0071     0.0277   0.2572    0.797   -0.0472   

                     Upper CI  
log_entry              0.2295  
log_incumbents         0.9102  
log_restriction_1_2    0.0614  
                     Parameter  Std. Err.   T-stat  P-value  Lower CI  \
log_entry               0.1658     0.0244   6.8016   0.0000    0.1180   
log_incumbents          0.8181     0.0408  20.0320   0.0000    0.7380   
log_restriction_1_3     0.0413     0.0117   3.5236   0.0004    0.0183   

                     Upper CI  
log_entry              0.2136  
log_incumbents         0.8981  
log_restriction_1_3    0.0642  
                     Parameter  Std. Err.   T-stat  P-value  Lower CI  \
log_entry               0.0609     0.0234   2.6022   0.0093    0.0150   
log_incumben

   2. all levels
   $$\log({\sf exit}_{naics_4,t}) = \alpha_s + \gamma_t + \beta_1 {\sf Reg}_{naics_2,t} + \beta_2 {\sf Reg}_{naics_3,t} + \beta_3 {\sf Reg}_{naics_4,t} + \beta_4 \log({\sf entry})  + \beta_5 \log({\sf incumbents}) + \epsilon_{naics_4,t}$$

In [34]:
data = df_age
# sample restriction
data = data[data.year > 1985]

# regression
data = data.set_index(['sector', 'year'])
data = data.loc[:, ['log_entry', 'log_death', 'log_incumbents', 
                    'log_restriction_1_2', 'log_restriction_1_3', 
                    'log_restriction_1_4']].dropna()
mod = PanelOLS.from_formula(formula = 
                            'log_death ~ log_restriction_1_2 + log_restriction_1_3 + log_restriction_1_4  \
                            + log_incumbents + log_entry + EntityEffects + TimeEffects',
                            data = data)
res = mod.fit(cov_type = 'heteroskedastic')

# results
results_as_html = res.summary.tables[1].as_html()
results_table = pd.read_html(results_as_html, header=0, index_col=0)[0]

print(results_table)

KeyError: "None of [Index(['log_entry', 'log_death', 'log_incumbents', 'log_restriction_1_2',\n       'log_restriction_1_3', 'log_restriction_1_4'],\n      dtype='object')] are in the [columns]"

Dep. Variable:,death_rate,R-squared:,0.0072
Estimator:,PanelOLS,R-squared (Between):,0.6091
No. Observations:,2176,R-squared (Within):,-0.0046
Date:,"Thu, May 05 2022",R-squared (Overall):,0.5579
Time:,14:02:16,Log-likelihood,4950.5
Cov. Estimator:,Robust,,
,,F-statistic:,3.7880
Entities:,262,P-value,0.0045
Avg Obs:,8.3053,Distribution:,"F(4,2075)"
Min Obs:,0.0000,,
Max Obs:,34.000,F-statistic (robust):,1.4800


## 4. Exit Rates and Reg Data conditional on Age

This section redoes the previous section analysis but additional conditional on age.

In [60]:
# create data at different aggregation
data_dic = dict()

# reg data
regdata = pd.read_csv('C:\\Users\\zach_\\Nobackupfile\\Data\\BDS\\Regdata\\regdata_4_0_industries.csv')
regdata['sector_reg'] = regdata['NAICS']
regdata = regdata.loc[:,["year", "sector_reg", "industry_restrictions_1_0", "industry_restrictions_2_0"]]

# load dataset
file_name = "bds2019_naics_4_age"
file_address = f"C:\\Users\\zach_\\Nobackupfile\\Data\\BDS\\{file_name}.csv"
df = pd.read_csv(file_address)

# clean data
df['firms'] = pd.to_numeric(df['firms'], errors='coerce', downcast=None)
df['death'] = pd.to_numeric(df['firmdeath_firms'], errors='coerce', downcast=None)

# creat aggregation for entry and death measure
df_age = pd.DataFrame()
df_age['entry_whole'] = df.loc[df['fage'] == 'a) 0', :].groupby(['year', 'sector'])['firms'].sum()
df_age['incumbents_whole'] = df.loc[df['fage'] != 'a) 0', :].groupby(['year', 'sector'])['firms'].sum()
df_age = df_age.reset_index()
# df_age['death'] = df.loc[df['fage'] != 'a) 0', :].groupby(['year', 'sector'])['firmdeath_firms'].sum()

# merge df and regdata
for naics in range(2, 5):
    # clean and create variables
    sector_name = f"sector_{naics}"
    df[sector_name] = df['sector'].astype(str).str.slice(0,naics)
    df[sector_name] = pd.to_numeric(df[sector_name])

    df = df.merge(regdata, how = "left", left_on=["year", sector_name], right_on=["year", "sector_reg"])
    df = df.rename(columns={"industry_restrictions_1_0": f"industry_restrictions_1_0_{naics}", "industry_restrictions_2_0": f"industry_restrictions_2_0_{naics}"}, errors="raise")
    df = df.drop(columns='sector_reg')
    df[f'log_restriction_1_{naics}'] = np.log(df[f'industry_restrictions_1_0_{naics}'])
    df[f'log_restriction_2_{naics}'] = np.log(df[f'industry_restrictions_2_0_{naics}'])

# merge with df_age
df = df.merge(df_age, how = "left", left_on=["year", "sector"], right_on=["year", "sector"])

# create some variables
df['log_entry_whole'] = np.log(df['entry_whole'])
df['log_death'] = np.log(df['death'])
df['log_incumbents_whole'] = np.log(df['incumbents_whole'])
df['entry_rate_whole'] = df['entry_whole']/df['incumbents_whole'] 
df['death_rate_age'] = df['death']/df['firms'] 

# create age dummy
# define age groups
df = df.sort_values(by=['year', 'sector', 'fage'])
i = 0
for fage in df.fage.unique():
    df.loc[df.fage == fage, 'age_grp_dummy'] = i
    i = i + 1

df.loc[df.age_grp_dummy >= 7, 'age_grp_dummy'] = 7

C:\Users\zach_\anaconda3\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning:

divide by zero encountered in log



1. Check the impacts of the level of aggregation
   1. one level reg only
   $${\sf Exit\_rate}_{naics_4, t} = \alpha_{naics_4} + \gamma_t + \kappa_{{\sf age}} + \beta_1 {\sf Reg}_{s,t}  + \epsilon_{naics_4, t}$$

In [61]:
results_dict_1 = dict()
for naics_curr in range(2, 5):
    # load data
    data = df
    # sample restriction
    data = data[data.year > 1985]

    # regression
    data = data.set_index(['sector', 'year'])
    data = data.loc[:, ['entry_rate_whole', 'death_rate_age', 'age_grp_dummy', 'log_incumbents_whole', f'log_restriction_1_{naics_curr}']].dropna()
    mod = PanelOLS.from_formula(formula = f'death_rate_age ~ log_restriction_1_{naics_curr} + C(age_grp_dummy) + EntityEffects + TimeEffects', data = data)
    res = mod.fit(cov_type = 'heteroskedastic')

    # results
    results_as_html = res.summary.tables[1].as_html()
    results_table = pd.read_html(results_as_html, header=0, index_col=0)[0]
    file_name_curr = f"reg_{naics_curr}"
    results_dict_1[file_name_curr] = results_table

print(results_dict_1['reg_2'])
print(results_dict_1['reg_3'])
print(results_dict_1['reg_4'])


                         Parameter  Std. Err.  T-stat  P-value  Lower CI  \
C(age_grp_dummy)[T.1.0]     0.1109     0.0251  4.4183   0.0000    0.0617   
C(age_grp_dummy)[T.2.0]     0.0279     0.0252  1.1100   0.2670   -0.0214   
C(age_grp_dummy)[T.3.0]    -0.0021     0.0251 -0.0821   0.9345   -0.0513   
C(age_grp_dummy)[T.4.0]    -0.0204     0.0252 -0.8087   0.4187   -0.0697   
C(age_grp_dummy)[T.5.0]    -0.0331     0.0252 -1.3145   0.1887   -0.0824   
C(age_grp_dummy)[T.6.0]    -0.0571     0.0252 -2.2683   0.0233   -0.1064   
C(age_grp_dummy)[T.7.0]    -0.0886     0.0251 -3.5265   0.0004   -0.1379   
log_restriction_1_2         0.0138     0.0024  5.7980   0.0000    0.0092   

                         Upper CI  
C(age_grp_dummy)[T.1.0]    0.1601  
C(age_grp_dummy)[T.2.0]    0.0773  
C(age_grp_dummy)[T.3.0]    0.0472  
C(age_grp_dummy)[T.4.0]    0.0290  
C(age_grp_dummy)[T.5.0]    0.0162  
C(age_grp_dummy)[T.6.0]   -0.0078  
C(age_grp_dummy)[T.7.0]   -0.0394  
log_restriction_1_2        

   2. all levels
   $${\sf Exit\_rate}_{naics_4, t} = \alpha_{naics_4} + \gamma_t + + \kappa_{{\sf age}} \beta_1 {\sf Reg}_{naics_2,t} + \beta_2 {\sf Reg}_{naics_3,t} + \beta_3 {\sf Reg}_{naics_4,t} + \epsilon_{naics_4, t}$$

In [63]:
data = df
# sample restriction
data = data[data.year > 1985]

# regression
data = data.set_index(['sector', 'year'])
data = data.loc[:, ['entry_rate_whole', 'death_rate_age', 'age_grp_dummy', 'log_incumbents_whole', 
                    'log_restriction_1_2', 'log_restriction_1_3', 
                    'log_restriction_1_4']].dropna()
mod = PanelOLS.from_formula(formula = 
                            'death_rate_age ~ log_restriction_1_2 + log_restriction_1_3 + log_restriction_1_4 + C(age_grp_dummy) \
                            + EntityEffects + TimeEffects',
                            data = data)
res = mod.fit(cov_type = 'heteroskedastic')

# results
results_as_html = res.summary.tables[1].as_html()
results_table = pd.read_html(results_as_html, header=0, index_col=0)[0]

print(results_table)


                         Parameter  Std. Err.  T-stat  P-value  Lower CI  \
C(age_grp_dummy)[T.1.0]     0.1951     0.0415  4.7011   0.0000    0.1137   
C(age_grp_dummy)[T.2.0]     0.1202     0.0417  2.8837   0.0039    0.0385   
C(age_grp_dummy)[T.3.0]     0.0942     0.0416  2.2646   0.0236    0.0127   
C(age_grp_dummy)[T.4.0]     0.0780     0.0416  1.8733   0.0610   -0.0036   
C(age_grp_dummy)[T.5.0]     0.0641     0.0416  1.5394   0.1237   -0.0175   
C(age_grp_dummy)[T.6.0]     0.0378     0.0416  0.9075   0.3642   -0.0438   
C(age_grp_dummy)[T.7.0]     0.0064     0.0415  0.1543   0.8774   -0.0750   
log_restriction_1_2        -0.0008     0.0033 -0.2516   0.8013   -0.0074   
log_restriction_1_3         0.0015     0.0031  0.4700   0.6384   -0.0047   
log_restriction_1_4         0.0052     0.0027  1.9089   0.0563   -0.0001   

                         Upper CI  
C(age_grp_dummy)[T.1.0]    0.2764  
C(age_grp_dummy)[T.2.0]    0.2020  
C(age_grp_dummy)[T.3.0]    0.1758  
C(age_grp_dummy)[T.

2. Impacts of Entry Rates
   1. one level of regulation
   $${\sf Exit\_rate} _{naics_4,t} = \alpha_s + \gamma_t + \kappa_{{\sf age}} + \beta_1 {\sf Reg}_{s,t} + \beta_2 {\sf entry\_rates} + \epsilon_{naics_4,t}$$

In [67]:
results_dict_2 = dict()
for naics_curr in range(2, 5):
    # load data
    data = df
    # sample restriction
    data = data[data.year > 1985]

    # regression
    data = data.set_index(['sector', 'year'])
    data = data.loc[:, ['entry_rate_whole', 'death_rate_age', 'age_grp_dummy', 'log_incumbents_whole', f'log_restriction_1_{naics_curr}']].dropna()
    mod = PanelOLS.from_formula(formula = f'death_rate_age ~ log_restriction_1_{naics_curr}  + entry_rate_whole + C(age_grp_dummy) + EntityEffects + TimeEffects',
                                data = data)
    res = mod.fit(cov_type = 'heteroskedastic')

    # results
    results_as_html = res.summary.tables[1].as_html()
    results_table = pd.read_html(results_as_html, header=0, index_col=0)[0]
    file_name_curr = f"reg_{naics_curr}"
    results_dict_2[file_name_curr] = results_table
    

In [68]:
print(results_dict_2['reg_2'])

                         Parameter  Std. Err.  T-stat  P-value  Lower CI  \
C(age_grp_dummy)[T.1.0]     0.1200     0.0254  4.7326   0.0000    0.0703   
C(age_grp_dummy)[T.2.0]     0.0371     0.0253  1.4624   0.1436   -0.0126   
C(age_grp_dummy)[T.3.0]     0.0071     0.0253  0.2798   0.7796   -0.0425   
C(age_grp_dummy)[T.4.0]    -0.0112     0.0253 -0.4425   0.6581   -0.0609   
C(age_grp_dummy)[T.5.0]    -0.0239     0.0253 -0.9452   0.3445   -0.0736   
C(age_grp_dummy)[T.6.0]    -0.0479     0.0253 -1.8919   0.0585   -0.0976   
C(age_grp_dummy)[T.7.0]    -0.0795     0.0253 -3.1408   0.0017   -0.1291   
entry_rate_whole            0.0775     0.0259  2.9902   0.0028    0.0267   
log_restriction_1_2         0.0122     0.0024  4.9790   0.0000    0.0074   

                         Upper CI  
C(age_grp_dummy)[T.1.0]    0.1697  
C(age_grp_dummy)[T.2.0]    0.0867  
C(age_grp_dummy)[T.3.0]    0.0567  
C(age_grp_dummy)[T.4.0]    0.0385  
C(age_grp_dummy)[T.5.0]    0.0257  
C(age_grp_dummy)[T.6.0]

In [69]:
print(results_dict_2['reg_3'])

                         Parameter  Std. Err.   T-stat  P-value  Lower CI  \
C(age_grp_dummy)[T.1.0]     0.1783     0.0100  17.8090   0.0000    0.1587   
C(age_grp_dummy)[T.2.0]     0.1036     0.0102  10.1390   0.0000    0.0836   
C(age_grp_dummy)[T.3.0]     0.0763     0.0102   7.4812   0.0000    0.0563   
C(age_grp_dummy)[T.4.0]     0.0576     0.0101   5.6822   0.0000    0.0378   
C(age_grp_dummy)[T.5.0]     0.0462     0.0102   4.5239   0.0000    0.0262   
C(age_grp_dummy)[T.6.0]     0.0204     0.0102   2.0060   0.0449    0.0005   
C(age_grp_dummy)[T.7.0]    -0.0105     0.0102  -1.0358   0.3003   -0.0305   
entry_rate_whole            0.0929     0.0295   3.1526   0.0016    0.0351   
log_restriction_1_3         0.0061     0.0011   5.6375   0.0000    0.0040   

                         Upper CI  
C(age_grp_dummy)[T.1.0]    0.1979  
C(age_grp_dummy)[T.2.0]    0.1237  
C(age_grp_dummy)[T.3.0]    0.0962  
C(age_grp_dummy)[T.4.0]    0.0775  
C(age_grp_dummy)[T.5.0]    0.0662  
C(age_grp_dum

In [70]:
print(results_dict_2['reg_4'])

                         Parameter  Std. Err.   T-stat  P-value  Lower CI  \
C(age_grp_dummy)[T.1.0]     0.1969     0.0104  18.9850   0.0000    0.1766   
C(age_grp_dummy)[T.2.0]     0.1266     0.0104  12.1720   0.0000    0.1062   
C(age_grp_dummy)[T.3.0]     0.1004     0.0102   9.8082   0.0000    0.0803   
C(age_grp_dummy)[T.4.0]     0.0817     0.0103   7.9584   0.0000    0.0616   
C(age_grp_dummy)[T.5.0]     0.0690     0.0102   6.7323   0.0000    0.0489   
C(age_grp_dummy)[T.6.0]     0.0438     0.0102   4.2905   0.0000    0.0238   
C(age_grp_dummy)[T.7.0]     0.0122     0.0102   1.1966   0.2315   -0.0078   
entry_rate_whole           -0.0995     0.0201  -4.9576   0.0000   -0.1389   
log_restriction_1_4         0.0057     0.0012   4.6220   0.0000    0.0033   

                         Upper CI  
C(age_grp_dummy)[T.1.0]    0.2173  
C(age_grp_dummy)[T.2.0]    0.1470  
C(age_grp_dummy)[T.3.0]    0.1204  
C(age_grp_dummy)[T.4.0]    0.1019  
C(age_grp_dummy)[T.5.0]    0.0890  
C(age_grp_dum

   2. all levels
   $${\sf exit\_rate} _{naics_4,t} = \alpha_s + \gamma_t + \kappa_{{\sf age}} + \beta_1 {\sf Reg}_{naics_2,t} + \beta_2 {\sf Reg}_{naics_3,t} + \beta_3 {\sf Reg}_{naics_4,t} + \beta_4 {\sf entry\_rates} + \epsilon_{naics_4,t}$$

In [72]:
data = df
# sample restriction
data = data[data.year > 1985]

# regression
data = data.set_index(['sector', 'year'])
data = data.loc[:, ['entry_rate_whole', 'death_rate_age',  'age_grp_dummy', 'log_incumbents_whole',
                    'log_restriction_1_2', 'log_restriction_1_3', 
                    'log_restriction_1_4']].dropna()
mod = PanelOLS.from_formula(formula = 
                            'death_rate_age ~ log_restriction_1_2 + log_restriction_1_3 + log_restriction_1_4  \
                            + entry_rate_whole + C(age_grp_dummy) + EntityEffects + TimeEffects',
                            data = data)
res = mod.fit(cov_type = 'heteroskedastic')

# results
results_as_html = res.summary.tables[1].as_html()
results_table = pd.read_html(results_as_html, header=0, index_col=0)[0]

print(results_table)

                         Parameter  Std. Err.  T-stat  P-value  Lower CI  \
C(age_grp_dummy)[T.1.0]     0.2049     0.0415  4.9339   0.0000    0.1235   
C(age_grp_dummy)[T.2.0]     0.1300     0.0417  3.1178   0.0018    0.0483   
C(age_grp_dummy)[T.3.0]     0.1040     0.0416  2.4984   0.0125    0.0224   
C(age_grp_dummy)[T.4.0]     0.0878     0.0416  2.1082   0.0350    0.0062   
C(age_grp_dummy)[T.5.0]     0.0739     0.0416  1.7744   0.0760   -0.0077   
C(age_grp_dummy)[T.6.0]     0.0475     0.0416  1.1413   0.2538   -0.0341   
C(age_grp_dummy)[T.7.0]     0.0162     0.0416  0.3897   0.6967   -0.0653   
entry_rate_whole           -0.0990     0.0204 -4.8501   0.0000   -0.1390   
log_restriction_1_2         0.0004     0.0034  0.1318   0.8951   -0.0061   
log_restriction_1_3        -0.0008     0.0031 -0.2711   0.7863   -0.0070   
log_restriction_1_4         0.0060     0.0027  2.2285   0.0259    0.0007   

                         Upper CI  
C(age_grp_dummy)[T.1.0]    0.2862  
C(age_grp_dummy

## 5. Generate Cohort Level Data

In [3]:
# create data at different aggregation
data_dic = dict()

# reg data
regdata = pd.read_csv('C:\\Users\\zach_\\Nobackupfile\\Data\\BDS\\Regdata\\regdata_4_0_industries.csv')
regdata['sector_reg'] = regdata['NAICS']
regdata = regdata.loc[:,["year", "sector_reg", "industry_restrictions_1_0", "industry_restrictions_2_0"]]

# load dataset
file_name = "bds2019_naics_4_age"
file_address = f"C:\\Users\\zach_\\Nobackupfile\\Data\\BDS\\{file_name}.csv"
df = pd.read_csv(file_address)

# clean data
df['firms'] = pd.to_numeric(df['firms'], errors='coerce', downcast=None)
df['death'] = pd.to_numeric(df['firmdeath_firms'], errors='coerce', downcast=None)

# creat aggregation for entry and death measure
df_age = pd.DataFrame()
df_age['entry_whole'] = df.loc[df['fage'] == 'a) 0', :].groupby(['year', 'sector'])['firms'].sum()
df_age['incumbents_whole'] = df.loc[df['fage'] != 'a) 0', :].groupby(['year', 'sector'])['firms'].sum()
df_age = df_age.reset_index()
# df_age['death'] = df.loc[df['fage'] != 'a) 0', :].groupby(['year', 'sector'])['firmdeath_firms'].sum()

# merge df and regdata
for naics in range(2, 5):
    # clean and create variables
    sector_name = f"sector_{naics}"
    df[sector_name] = df['sector'].astype(str).str.slice(0,naics)
    df[sector_name] = pd.to_numeric(df[sector_name])

    df = df.merge(regdata, how = "left", left_on=["year", sector_name], right_on=["year", "sector_reg"])
    df = df.rename(columns={"industry_restrictions_1_0": f"industry_restrictions_1_0_{naics}", "industry_restrictions_2_0": f"industry_restrictions_2_0_{naics}"}, errors="raise")
    df = df.drop(columns='sector_reg')
    df[f'log_restriction_1_{naics}'] = np.log(df[f'industry_restrictions_1_0_{naics}'])
    df[f'log_restriction_2_{naics}'] = np.log(df[f'industry_restrictions_2_0_{naics}'])

# merge with df_age
df = df.merge(df_age, how = "left", left_on=["year", "sector"], right_on=["year", "sector"])

# merge with sector gdp
gdp = pd.read_csv('C:\\Users\\zach_\\Nobackupfile\\Data\\BDS\\BEA\\gdp.csv')
gdp = pd.melt(gdp, id_vars='sector_2', var_name='year', value_name='gdp')
gdp['year'] = pd.to_numeric(gdp['year']).astype(np.int64)
df = df.merge(gdp, how = "left", left_on=["year", "sector_2"], right_on=["year", "sector_2"])

# create some variables
df['log_entry_whole'] = np.log(df['entry_whole'])
df['log_death'] = np.log(df['death'])
df['log_incumbents_whole'] = np.log(df['incumbents_whole'])
df['entry_rate_whole'] = df['entry_whole']/df['incumbents_whole'] 
df['death_rate_age'] = df['death']/df['firms'] 
df['log_gdp'] = np.log(df['gdp'])
# create age dummy
# define age groups
df = df.sort_values(by=['year', 'sector', 'fage'])
i = 0
for fage in df.fage.unique():
    df.loc[df.fage == fage, 'age_grp_dummy'] = i
    i = i + 1

df.loc[df.age_grp_dummy >= 7, 'age_grp_dummy'] = 7

c:\Users\zach_\.virtualenvs\RFE_project-5yjSZlxr\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning:

divide by zero encountered in log

c:\Users\zach_\.virtualenvs\RFE_project-5yjSZlxr\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning:

divide by zero encountered in log

c:\Users\zach_\.virtualenvs\RFE_project-5yjSZlxr\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning:

divide by zero encountered in log



In [4]:
# create cohort level data
df.loc[df.age_grp_dummy <= 5, 'cohort'] = \
    df.loc[df.age_grp_dummy <= 5, 'year'] - df.loc[df.age_grp_dummy <= 5, 'age_grp_dummy']

# merge reg data at entry year
regdata = regdata.rename(columns={"year": "cohort"})

for naics in range(2, 5):
    # clean and create variables
    sector_name = f"sector_{naics}"

    df = df.merge(regdata, how = "left", left_on=["cohort", sector_name], right_on=["cohort", "sector_reg"])
    df = df.rename(columns={"industry_restrictions_1_0": f"industry_restrictions_1_0_{naics}_cohort", 
                            "industry_restrictions_2_0": f"industry_restrictions_2_0_{naics}_cohort"}, 
                            errors="raise")
                            
    df = df.drop(columns='sector_reg')
    df[f'log_restriction_1_{naics}_cohort'] = np.log(df[f'industry_restrictions_1_0_{naics}_cohort'])
    df[f'log_restriction_2_{naics}_cohort'] = np.log(df[f'industry_restrictions_2_0_{naics}_cohort'])

# merge with sector gdp
gdp = pd.read_csv('C:\\Users\\zach_\\Nobackupfile\\Data\\BDS\\BEA\\gdp.csv')
gdp = pd.melt(gdp, id_vars='sector_2', var_name='cohort', value_name='gdp_cohort')
gdp['cohort'] = pd.to_numeric(gdp['cohort']).astype(np.int64)
df = df.merge(gdp, how = "left", left_on=["cohort", "sector_2"], right_on=["cohort", "sector_2"])


# merge the entry rate at the cohort level
df_age_new = df_age.rename(columns={"year": "cohort", 
                                    "entry_whole": "entry_whole_cohort",
                                    "incumbents_whole": "incumbents_whole_cohort"})
df_age_new = df_age_new[["cohort", "sector", "entry_whole_cohort", "incumbents_whole_cohort"]]
df = df.merge(df_age_new, 
    how = "left", left_on=["cohort", 'sector'], right_on=["cohort", "sector"])
df["entry_rate_whole_cohort"] = df["entry_whole_cohort"]/df["incumbents_whole_cohort"]
df["log_gdp_cohort"] = np.log(df["gdp_cohort"])

In [5]:
df.fage.unique()

array(['a) 0', 'b) 1', 'c) 2', 'd) 3', 'e) 4', 'f) 5', 'g) 6 to 10',
       'h) 11 to 15', 'i) 16 to 20', 'j) 21 to 25', 'k) 26+',
       'l) Left Censored'], dtype=object)

1. Check the impacts of the level of aggregation
   1. one level reg only
   $${\sf Exit\_rate}_{naics_4, cohort, age} = \alpha_{naics_4} + \gamma_{cohort} + \kappa_{{age}} + \beta_1 {\sf Reg}_{s,cohort} +\beta_2 {\sf Reg}_{s,cohort + age} + \epsilon_{naics_4, cohort, age}$$

In [25]:
results_dict_1 = dict()
for naics_curr in range(2, 5):
    # load data
    data = df
    # sample restriction
    data = data[data.year > 1985]
    data = data[data.age_grp_dummy <= 5]
    
    # regression
    data = data.set_index(['sector', 'cohort'])
    data = data.loc[:, ['entry_rate_whole', 'death_rate_age', 'age_grp_dummy', 'year', 'log_incumbents_whole', 'log_gdp', 'log_gdp_cohort',
                        f'log_restriction_2_{naics_curr}', f'log_restriction_2_{naics_curr}_cohort']].dropna()
    mod = PanelOLS.from_formula(formula = f'death_rate_age ~ log_restriction_2_{naics_curr} + log_restriction_2_{naics_curr}_cohort \
                                            + C(age_grp_dummy) + EntityEffects + TimeEffects', data = data, drop_absorbed=True)
    res = mod.fit(cov_type = 'heteroskedastic')

    # results
    file_name_curr = f"reg_{naics_curr}"
    results_dict_1[file_name_curr] =  res.summary

"""
    results_as_html = res.summary.tables[1].as_html()
    results_table = pd.read_html(results_as_html, header=0, index_col=0)[0]
    file_name_curr = f"reg_{naics_curr}"
"""

'\n    results_as_html = res.summary.tables[1].as_html()\n    results_table = pd.read_html(results_as_html, header=0, index_col=0)[0]\n    file_name_curr = f"reg_{naics_curr}"\n'

In [36]:
results_dict_1['reg_2'].as_csv()

'                     PanelOLS Estimation Summary                     \nDep. Variable:   ,death_rate_age  ,  R-squared:           ,0.2595    \nEstimator:       ,PanelOLS        ,  R-squared (Between): ,0.0405    \nNo. Observations:,27205           ,  R-squared (Within):  ,0.2597    \nDate:            ,Wed, May 25 2022,  R-squared (Overall): ,0.2325    \nTime:            ,17:04:07        ,  Log-likelihood       ,2.848e+04 \nCov. Estimator:  ,Robust          ,                       ,          \n                 ,                ,  F-statistic:         ,1576.8    \nEntities:        ,262             ,  P-value              ,0.0000    \nAvg Obs:         ,103.84          ,  Distribution:        ,F(6,26997)\nMin Obs:         ,0.0000          ,                       ,          \nMax Obs:         ,180.00          ,  F-statistic (robust):,1185.8    \n                 ,                ,  P-value              ,0.0000    \nTime periods:    ,38              ,  Distribution:        ,F(6,26997)\nAvg O

In [19]:
results_table = pd.read_html(results_dict_1['reg_2'], header=0, index_col=0)

In [24]:
results_table[0].to_markdown(buf="C:/Users/zach_/Nobackupfile/Data/BDS/test.md")

In [127]:
print(results_dict_1['reg_3'])

                            Parameter  Std. Err.  T-stat  P-value  Lower CI  \
C(age_grp_dummy)[T.1.0]        0.1622     0.0383  4.2325   0.0000  0.087100   
C(age_grp_dummy)[T.2.0]        0.0783     0.0387  2.0210   0.0433  0.002400   
C(age_grp_dummy)[T.3.0]        0.0495     0.0387  1.2802   0.2005 -0.026300   
C(age_grp_dummy)[T.4.0]        0.0319     0.0387  0.8258   0.4089 -0.043900   
C(age_grp_dummy)[T.5.0]        0.0196     0.0388  0.5048   0.6137 -0.056400   
log_restriction_2_3            0.0094     0.0048  1.9427   0.0521 -0.000084   
log_restriction_2_3_cohort     0.0007     0.0051  0.1299   0.8966 -0.009300   

                            Upper CI  
C(age_grp_dummy)[T.1.0]       0.2373  
C(age_grp_dummy)[T.2.0]       0.1542  
C(age_grp_dummy)[T.3.0]       0.1253  
C(age_grp_dummy)[T.4.0]       0.1077  
C(age_grp_dummy)[T.5.0]       0.0955  
log_restriction_2_3           0.0189  
log_restriction_2_3_cohort    0.0106  


In [128]:
print(results_dict_1['reg_4'])

                            Parameter  Std. Err.  T-stat  P-value  Lower CI  \
C(age_grp_dummy)[T.1.0]        0.1798     0.0332  5.4220   0.0000    0.1148   
C(age_grp_dummy)[T.2.0]        0.1046     0.0333  3.1409   0.0017    0.0393   
C(age_grp_dummy)[T.3.0]        0.0775     0.0333  2.3299   0.0198    0.0123   
C(age_grp_dummy)[T.4.0]        0.0616     0.0333  1.8480   0.0646   -0.0037   
C(age_grp_dummy)[T.5.0]        0.0481     0.0334  1.4410   0.1496   -0.0173   
log_restriction_2_4            0.0024     0.0049  0.4889   0.6249   -0.0072   
log_restriction_2_4_cohort     0.0051     0.0051  1.0042   0.3153   -0.0048   

                            Upper CI  
C(age_grp_dummy)[T.1.0]       0.2448  
C(age_grp_dummy)[T.2.0]       0.1699  
C(age_grp_dummy)[T.3.0]       0.1427  
C(age_grp_dummy)[T.4.0]       0.1270  
C(age_grp_dummy)[T.5.0]       0.1134  
log_restriction_2_4           0.0119  
log_restriction_2_4_cohort    0.0150  


2. Check the impacts of entry rates
   $${\sf Exit\_rate}_{naics_4, cohort, age} = \alpha_{naics_4} + \gamma_{cohort} + \kappa_{{age}} + \beta_1 {\sf Reg}_{s,cohort} +\beta_2 {\sf Reg}_{s,cohort + age} + \beta_3 {\sf Entry\_rate}_{s,cohort} + \beta_4 {\sf Entry\_rate}_{s,cohort + age} + \epsilon_{naics_4, cohort, age}$$

In [182]:
results_dict_1 = dict()
for naics_curr in range(2, 5):
    # load data
    data = df
    # sample restriction
    data = data[data.year > 1985]
    data = data[data.age_grp_dummy <= 5]
    
    # regression
    data = data.set_index(['sector', 'cohort'])
    data = data.loc[:, ['entry_rate_whole', 'death_rate_age', 'age_grp_dummy', 'year', 'log_incumbents_whole', 'entry_rate_whole_cohort', 'log_gdp', 'log_gdp_cohort',
                        f'log_restriction_2_{naics_curr}', f'log_restriction_2_{naics_curr}_cohort']].dropna()
    mod = PanelOLS.from_formula(formula = f'death_rate_age ~ log_restriction_2_{naics_curr} + log_restriction_2_{naics_curr}_cohort  \
                                            + C(age_grp_dummy)   + EntityEffects + TimeEffects', data = data, drop_absorbed=True)
    res = mod.fit(cov_type = 'heteroskedastic')

    # results
    results_as_html = res.summary.tables[0].as_html()
    results_table = pd.read_html(results_as_html, header=0, index_col=0)[0]
    file_name_curr = f"reg_{naics_curr}"
    results_dict_1[file_name_curr] = results_table


In [183]:
np.std(df['log_restriction_2_2'])

1.1105038859330154

In [184]:
print(results_dict_1['reg_2'])

                     death_rate_age             R-squared:      0.2595
Dep. Variable:                                                        
Estimator:                 PanelOLS   R-squared (Between):      0.0409
No. Observations:             27204    R-squared (Within):      0.2597
Date:              Mon, May 16 2022   R-squared (Overall):      0.2324
Time:                      15:48:33         Log-likelihood   2.848e+04
Cov. Estimator:              Robust                    NaN         NaN
NaN                             NaN           F-statistic:      1576.8
Entities:                       262                P-value      0.0000
Avg Obs:                     103.83          Distribution:  F(6,26996)
Min Obs:                     0.0000                    NaN         NaN
Max Obs:                     180.00  F-statistic (robust):      1185.5
NaN                             NaN                P-value      0.0000
Time periods:                    38          Distribution:  F(6,26996)
Avg Ob

In [175]:
print(results_dict_1['reg_3'])

                     death_rate_age             R-squared:      0.2551
Dep. Variable:                                                        
Estimator:                 PanelOLS   R-squared (Between):      0.2009
No. Observations:             16780    R-squared (Within):      0.2621
Date:              Mon, May 16 2022   R-squared (Overall):      0.2549
Time:                      15:45:34         Log-likelihood   1.709e+04
Cov. Estimator:              Robust                    NaN         NaN
NaN                             NaN           F-statistic:      813.76
Entities:                       262                P-value      0.0000
Avg Obs:                     64.046          Distribution:  F(7,16634)
Min Obs:                     0.0000                    NaN         NaN
Max Obs:                     180.00  F-statistic (robust):      688.53
NaN                             NaN                P-value      0.0000
Time periods:                    38          Distribution:  F(7,16634)
Avg Ob

In [36]:
print(results_dict_1['reg_4'])

                            Parameter  Std. Err.  T-stat  P-value  Lower CI  \
C(age_grp_dummy)[T.1.0]        0.1971     0.0466  4.2265   0.0000    0.1057   
C(age_grp_dummy)[T.2.0]        0.1207     0.0468  2.5772   0.0100    0.0289   
C(age_grp_dummy)[T.3.0]        0.0935     0.0470  1.9887   0.0468    0.0013   
C(age_grp_dummy)[T.4.0]        0.0768     0.0472  1.6271   0.1037   -0.0157   
C(age_grp_dummy)[T.5.0]        0.0626     0.0474  1.3223   0.1861   -0.0302   
entry_rate_whole              -0.3136     0.0433 -7.2490   0.0000   -0.3984   
entry_rate_whole_cohort        0.2402     0.0461  5.2110   0.0000    0.1499   
log_gdp                       -0.0031     0.0053 -0.5808   0.5614   -0.0134   
log_restriction_2_4            0.0038     0.0049  0.7658   0.4438   -0.0059   
log_restriction_2_4_cohort     0.0048     0.0051  0.9312   0.3518   -0.0053   

                            Upper CI  
C(age_grp_dummy)[T.1.0]       0.2885  
C(age_grp_dummy)[T.2.0]       0.2125  
C(age_grp_dum

3. all levels

In [137]:
data = df
# sample restriction
data = data[data.year > 1985]

# regression
data = data.set_index(['sector', 'cohort'])
data = data.loc[:, ['entry_rate_whole', 'death_rate_age', 'age_grp_dummy', 'year', 'log_incumbents_whole', 'entry_rate_whole_cohort', 'log_gdp', 'log_gdp_cohort',
                    'log_restriction_2_2', 'log_restriction_2_3', 'log_restriction_2_4',
                    'log_restriction_2_2_cohort', 'log_restriction_2_3_cohort', 'log_restriction_2_4_cohort']].dropna()
mod = PanelOLS.from_formula(formula = f'death_rate_age ~ log_restriction_2_2_cohort + log_restriction_2_3_cohort + log_restriction_2_4_cohort \
                                        + log_restriction_2_2 + log_restriction_2_3 + log_restriction_2_4 + log_gdp + log_gdp_cohort\
                                        + C(age_grp_dummy) + entry_rate_whole_cohort + entry_rate_whole + EntityEffects + TimeEffects', data = data, drop_absorbed=True)
res = mod.fit(cov_type = 'heteroskedastic')

# results
results_as_html = res.summary.tables[1].as_html()
results_table = pd.read_html(results_as_html, header=0, index_col=0)[0]

print(results_table)

                            Parameter  Std. Err.  T-stat  P-value  Lower CI  \
C(age_grp_dummy)[T.1.0]        0.3560     0.0981  3.6288   0.0003    0.1637   
C(age_grp_dummy)[T.2.0]        0.2840     0.0984  2.8850   0.0039    0.0910   
C(age_grp_dummy)[T.3.0]        0.2591     0.0986  2.6284   0.0086    0.0659   
C(age_grp_dummy)[T.4.0]        0.2454     0.0990  2.4801   0.0132    0.0514   
C(age_grp_dummy)[T.5.0]        0.2338     0.0992  2.3576   0.0184    0.0394   
entry_rate_whole              -0.3002     0.0438 -6.8547   0.0000   -0.3860   
entry_rate_whole_cohort        0.2399     0.0468  5.1262   0.0000    0.1482   
log_gdp                       -0.0706     0.0119 -5.9151   0.0000   -0.0940   
log_gdp_cohort                 0.0623     0.0103  6.0428   0.0000    0.0421   
log_restriction_2_2            0.0198     0.0084  2.3696   0.0178    0.0034   
log_restriction_2_2_cohort    -0.0358     0.0095 -3.7605   0.0002   -0.0545   
log_restriction_2_3           -0.0057     0.0061 -0.

## 6. Decomposition

This section compute the decomposition of changes in firm death rates
$$\delta_t - \delta_0 =  \sum_i s_{t, 0} (\delta_{i, t} - \delta_{i, 0})  + \sum_i \delta_{i,0} (s_{i, t} - s_{i,0}) + \sum_i (s_{i, t} - s_{i,0}) (\delta_{i, t} - \delta_{i, 0})$$

In [77]:
# define decomposition function
def de_com(data, group):
    # clean and construct variables
    data['firms'] = pd.to_numeric(data['firms'], errors='coerce', downcast=None)
    data['firmdeath_firms'] = pd.to_numeric(data['firmdeath_firms'], errors='coerce', downcast=None)
    data['firm_death_rate'] = data['firmdeath_firms']/ data['firms']
    data['year'] = pd.to_numeric(data['year'], errors='coerce', downcast=None)

    # age group measure
    data['entry'] = (data['fage'] == 'b) 1')
    data['new'] = (data['fage'] == 'c) 2')|(data['fage'] == 'd) 3')|(data['fage'] == 'e) 4')|(data['fage'] == 'f) 5') 
    data['young'] = (data['fage'] == 'g) 6 to 10')
    data['old']  = (data['fage'] == 'h) 11 to 15')|(data['fage'] == 'i) 16 to 20')|(data['fage'] == 'j) 21 to 25')|(data['fage'] == 'k) 26+')|(data['fage'] == 'l) Left Censored') 
    data['age_gp'] = data['entry']* 1 + data['new']*2 + data['young'] *3 + data['old'] *4

    # sample restriction
    data = data[data['age_gp'] != 0]

    # decomposition
    death_0 = data[(data['year']>1985)&(data['year']<1991) ].groupby(group)['firmdeath_firms'].sum() / \
                                data[(data['year']>1985)&(data['year']<1991) ].groupby(group)['firms'].sum()
    death_1 = data[(data['year']>2014)&(data['year']<2020) ].groupby(group)['firmdeath_firms'].sum() / \
                                data[(data['year']>2014)&(data['year']<2020) ].groupby(group)['firms'].sum()
    share_0 = data[(data['year']>1985)&(data['year']<1991) ].groupby(group)['firms'].sum() / \
                                data.loc[(data['year']>1985)&(data['year']<1991), 'firms'].sum() 
    share_1 = data[(data['year']>2014)&(data['year']<2020) ].groupby(group)['firms'].sum() / \
                                data.loc[(data['year']>2014)&(data['year']<2020), 'firms'].sum() 
    death_total_0 = data.loc[(data['year']>1985)&(data['year']<1991), 'firmdeath_firms'].sum() / \
                                data.loc[(data['year']>1985)&(data['year']<1991), 'firms'].sum()
    death_total_1 = data.loc[(data['year']>2014)&(data['year']<2020), 'firmdeath_firms'].sum() / \
                                data.loc[(data['year']>2014)&(data['year']<2020), 'firms'].sum()    

    within_effects = (death_1 - death_0) @ share_0
    bwtween_effects = (share_1 - share_0) @ death_0
    cov_component = (death_1 - death_0) @ (share_1 - share_0)     
    total_effects = death_total_1 - death_total_0
    
    output = f"{total_effects:.5f}(Total) = {within_effects:.5f}(Within) + {bwtween_effects:.5f}(Between) + {cov_component:.5f}(Cov)"
    print(output)

In [78]:
# read bds data
data = pd.read_csv('C:\\Users\\zach_\\Nobackupfile\\Data\\BDS\\bds2019_firm_age.csv')
group = ['age_gp']
de_com(data, group)

-0.02765(Total) = -0.01674(Within) + -0.01373(Between) + 0.00282(Cov)


In [79]:
data = pd.read_csv('C:\\Users\\zach_\\Nobackupfile\\Data\\BDS\\bds2019_naics_2_age.csv')
group =  ['age_gp','sector']
de_com(data, group)

-0.02685(Total) = -0.01653(Within) + -0.01298(Between) + 0.00266(Cov)


In [66]:
data = pd.read_csv('C:\\Users\\zach_\\Nobackupfile\\Data\\BDS\\bds2019_firm_age.csv')


In [67]:
data

,year,fage,firms,estabs,emp,denom,estabs_entry,estabs_entry_rate,estabs_exit,estabs_exit_rate,job_creation,job_creation_births,job_creation_continuers,job_creation_rate_births,job_creation_rate,job_destruction,job_destruction_deaths,job_destruction_continuers,job_destruction_rate_deaths,job_destruction_rate,net_job_creation,net_job_creation_rate,reallocation_rate,firmdeath_firms,firmdeath_estabs,firmdeath_emp
0,1978,a) 0,485415,493592,2578695,1289365,493592,199.998,(X),(X),2578697,2578695,(X),199.997,199.998,(X),(X),(X),(X),(X),2578664,199.995,(X),(X),(X),(X)
1,1978,b) 1,355497,367372,2401016,2385945,4831,1.131,124648,29.172,824119,57161,766958,2.396,34.541,782252,526589,255663,22.07,32.786,41867,1.755,65.572,98669,99466,422139
2,1978,c) 2,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
3,1978,d) 3,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
4,1978,e) 4,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,2019,h) 11 to 15,691935,759764,9301145,9361423,24276,3.116,63095,8.098,1081116,190669,890447,2.037,11.549,1203106,381950,821156,4.08,12.852,-121990,-1.303,23.097,51605,52872,313640
500,2019,i) 16 to 20,488746,583630,9113140,9138328,17120,2.873,41483,6.962,943417,175537,767880,1.921,10.324,996772,295671,701101,3.236,10.908,-53355,-0.584,20.647,32472,33745,221819
501,2019,j) 21 to 25,401165,522686,9810797,9859779,13877,2.602,35225,6.604,884950,147254,737696,1.493,8.975,984760,261469,723291,2.652,9.988,-99810,-1.012,17.951,25258,26550,166257
502,2019,k) 26+,782174,1241854,28803062,28732919,39322,3.126,71263,5.666,2665371,553477,2111894,1.926,9.276,2541990,661479,1880511,2.302,8.847,123381,0.429,17.694,46543,49639,313472


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\zach_\anaconda3\python.exe -m pip install --upgrade pip' command.
